In [2]:
import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches, check_array
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.utils.validation import FLOAT_DTYPES
import numpy as np
from numpy.linalg import norm,inv,matrix_rank, pinv
# --------------------
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
import datetime
import Code.SIMPLS
import Code.ISIMPLS
# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [3]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]

n_fold = 74
test_size=30
n_comp_max=100
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)

4412


In [ ]:
def Read_Results(method):
    with h5py.File(f"./Results/PLS2_merra2_{method}.h5", "r") as f:
        dat={'timer': f['timer'][:],
             'coef':  f['coef'][:]}
    return dat


In [ ]:
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS
n_comp=63

dat = np.load(f'./data/wrk_ISIMPLS/ISIMPLS_fold73.npz')
ISIMPLS_RD = ISIMPLS(n_comp)

ISIMPLS_RD.W = dat['W'][:,:n_comp]
ISIMPLS_RD._x_mean_ = dat['x_mean']
ISIMPLS_RD._y_mean_ = dat['y_mean']
ISIMPLS_RD.n = dat['n']

tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    if (i%5==0):
        print(f"i={i}")
    if (i!=n_fold):
        continue
    X = X_train[train_index] - ISIMPLS_RD._x_mean_ 
    Y = Y_train[train_index] - ISIMPLS_RD._y_mean_
    ISIMPLS_RD.S = np.dot(X.T, Y)
    ISIMPLS_RD.C = np.dot(X.T, X)
    
ISIMPLS_RD.fit(X_train[test_index], Y_train[test_index])
ISIMPLS_RD._comp_coef(n_comp)
ISIMPLS_RD.coef_

In [ ]:
ISIMPLS = Read_Results('ISIMPLS')
print(RMSE(ISIMPLS_RD.coef_, ISIMPLS['coef']))

In [ ]:
def gen_slice_month(tim_st,tim_ed):
    tim = datetime.datetime.strptime(tim_st, "%Y%m%d")
    date_ed = datetime.datetime.strptime(  tim_ed, "%Y%m%d")
    ind_st = 0
    batches  = []
    trainset = []
    samples  = []
    count = 0
    while (tim < date_ed ):
        Year = tim.strftime("%Y")
        Month = tim.strftime("%m")
        if Month == '12':
            NextYYMM = "{:04d}01".format(int(tim.strftime("%Y"))+1)
        else:
            NextYYMM = Year+"{:02d}".format(int(tim.strftime("%m"))+1)
        tmp = datetime.datetime.strptime(  NextYYMM+"01" , "%Y%m%d")
        if (tmp > date_ed):
            tmp = date_ed
        days = (tmp - tim ).days
        ind_ed = ind_st + days
        batches.append(slice(ind_st,ind_ed))
        trainset.append(slice(ind_ed))
        samples.append(ind_ed)
        ind_st = ind_ed
        tim += datetime.timedelta(days=days)
    return batches,trainset,samples

In [ ]:
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

tim_st = '20110401'
tim_ed = '20230430'
batches, trainset, samples = gen_slice_month(tim_st,tim_ed)
train_ind = trainset[-1]

SIMPLS_RD = SIMPLS(n_components=63)
SIMPLS_RD.fit(X_train[train_ind], Y_train[train_ind])
SIMPLS_RD._comp_coef(n_comp)

SIMPLS = Read_Results('SIMPLS')
print(RMSE(SIMPLS_RD.coef_, SIMPLS['coef']))